<img src="imgs/qiskit.png" alt="QisKit" style="width: 50px;" align="right"/>

# Shor's Algorithm

<hr>

## Teach Me Qiskit

In [1]:
import random

try:
    import qiskit
except: # if qiskit is not installed
    print("Installing Qiskit")
    !pip install qiskit

### Prime Numbers

<hr>

<img src="imgs/primes.png" alt="Prime Visual - Wikimedia Commons" style="width: 300px;"/>

<br>

|  Composite numbers  |  Prime numbers  |
|:---:|:---:|
|  1  |     |
|     |  2  |
|     |  3  |
|  4  |     |
|     |  5  |
|  6  |     |
|     |  7  |
|  8  |     |
|  9  |     |



In [2]:
# https://stackoverflow.com/questions/11619942/print-series-of-prime-numbers-in-python

for num in range(2, 10):
    if all(num % i != 0 for i in range(2, num)):
        print(num)

2
3
5
7


### RSA Encryption

<hr>

<img src="imgs/rsa.jpg" alt="Prime Visual - Wikimedia Commons" style="width: 500px;"/>

Multiplying two numbers is simple

In [3]:
print(f"2 x 3 = {2 * 3}")

2 x 3 = 6


However, determining two numbers which multiplied to produce a certain number is much harder

In [4]:
# https://www.geeksforgeeks.org/find-two-distinct-prime-numbers-with-given-product/

def Sieve_of_Eratosthenes(n, isPrime) : 
    isPrime[0], isPrime[1] = False, False
    for i in range(2, n + 1) : 
        isPrime[i] = True
    for p in range(2, int(n**.5) + 1) : 
        if isPrime[p] == True:
            for i in range(p * 2, n + 1, p):
                isPrime[i] = False

def find_primes(n):
    isPrime = [False] * (n + 1)
    Sieve_of_Eratosthenes(n, isPrime)
    for i in range(2, n):
        x = int(n / i)
        if (isPrime[i] & isPrime[x] and x != i and x * i == n):
            print(f"{i} x {x} = {n}")
            break

In [5]:
find_primes(6)

2 x 3 = 6


### Shor's Algorithm

<hr>

<img src="imgs/shors.svg" alt="Prime Visual - Wikimedia Commons" style="width: 400px;"/>

Shor's Algorithm quickly determines two distinct prime numbers which multiply to produce a certain number.

1) Let's use 2 and 3 as our two distinct prime numbers

In [6]:
p = 2
q = 3

2) Now we can define our public key as the product of these two primes

In [7]:
key = p * q
print("Public key: ", key)

Public key:  6


3) We now need to encrypt our secret message, "I love Qiskit!"

In [8]:
string = "I love Qiskit!"